In [2]:
import numpy as np
import pandas as pd
import re

In [3]:
df=pd.read_csv("NYC_gyno_final.csv",header='infer')#None)
#df.columns=["Malpractice","Education","Name","Specialty"]

In [4]:
def split_out_year(ed):
    try:
        s=re.search(',\n',ed).span()
    except:
        s=(None,None)
    school=ed[0:s[0]]
    year= ed[s[1]:]
    return year
def split_out_school(ed):
    try:
        s=re.search(',\n',ed).span()
    except:
        s=(None,None)
    return ed[0:s[0]]
df['School']=list(map(split_out_school,df.Education))
df['Year']=list(map(split_out_year,df.Education))
df=df.drop("Education",axis=1)
df

,Malpractice,Name,Field,School,Year
0,Malpractice\nLast Updated 11-14-2013\nTO TOP\n...,"ABRAMS, JANET",Obstetrics and Gynecology,"FIRST TASHKENT STATE MED INST, TASKENT, UZBEKI...",1992
1,None reported,"ABU-RUSTUM, NADEEM",Obstetrics and Gynecology/Gynecologic Oncology,"AMERICAN UNIVERSITY OF BEIRUT, FAC OF MEDICINE...",1990
2,None reported,"AGUSTIN, ELMER",Obstetrics and Gynecology,"UNIVERSITY OF THE PHILIPPINES, COLLEGE OF MEDI...",1980
3,Malpractice\nLast Updated 02-22-2016\nTO TOP\n...,"AHRAM, JOSEPH",Obstetrics and Gynecology,"THE HEBREW UNIVERSITY, HADASSAH MEDICAL SCHOOL...",1978
4,None reported,"AL-IBRAHEEMI, ZAINAB",Obstetrics and Gynecology/Maternal-Fetal Medicine,"UNIVERSITY OF BAGHDAD, COLLEGE OF MEDICINE, BA...",2006
5,None reported,"ALLMEN, TARA",Obstetrics and Gynecology,"STONY BROOK UNIVERSITY HEALTH SCIENCES CENTER,...",1990
6,None reported,"AMARU, REBECCA",Obstetrics and Gynecology,"ICAHN SCH OF MED AT MT SINAI, NEW YORK, NY",1999
7,None reported,"AMRANE, SELMA",Obstetrics and Gynecology/Reproductive Endocri...,"UNIVERSITY OF MARYLAND SCHOOL OF MEDICINE, BAL...",2013
8,None reported,"ANDERSON, LISA",Obstetrics and Gynecology/Gynecologic Oncology,"UNIVERSITY DI BOLOGNA, FAC DI MED E CHIRURGIA,...",1980
9,Malpractice\nLast Updated 06-26-2017\nTO TOP\n...,"ANDRADES, JOANNE",Obstetrics and Gynecology,"SUNY UPSTATE MEDICAL UNIVERSITY, SYRACUSE NY",2006


In [5]:
us_states=pd.read_csv("States.csv")
us_states=us_states[us_states.x!='US']
us_states=us_states['x']

In [6]:
df['State'] = df['School'].str.extract('( [A-Z]{2}$)')
df.State=df.State.str.strip()

In [7]:
# Finding country where they went to school
ind=-1
countries=[]
for st in df.State:
    ind+=1
    if st in us_states.values:
        countries.append('USA')
    else:
        try:
            country=re.search(', [A-Z]+$',df.School[ind]).group() #{1}[a-z]
            #print(country[2:])
            countries.append(country[2:])#df['School'].str.extract(', [A-Z]{1}[a-z]+$')
        except Exception as e:
            #print(e)
            countries.append('')
            continue
df['Country']=countries

In [8]:
## TO DO ### Fix New York - NY
#df[df['Country']==''].School

In [9]:
def find_num_pay(mal):
    # check Judgments and Arbitration Awards
    start,end =re.search('Number of awards:',mal).span()
    try:
        a= int(mal[end+1:end+2])
    except:
        # check Settlements
        start,end_p =re.search('Number of payments:',mal).span()
        a= int(mal[end_p+1:end_p+2])
    return a

In [10]:
ind=0
num_payments=[]
for m in range(len(df)):
    #print(ind)
    if df.Malpractice[ind]=='None reported':
        num_payments.append(0)
    else:
        a=find_num_pay(df.Malpractice[ind])
        num_payments.append(a)
    ind+=1
df['Num_Payments']=num_payments

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
# test=pd.DataFrame([df.loc[8]],columns=df.columns)
# test.date='01-09-1998'
# df.loc[8,]

In [11]:
# df_try=df.loc[11]
# df.append([df_try]*5,ignore_index=True)

#create empty columns
df['ind_law_suit']=df['Num_Payments']
df['date']=df['Num_Payments']
df['area']=df['Num_Payments']
df['significance']=df['Num_Payments']

In [12]:
sum(df.Num_Payments)

71

In [13]:
def extract_law_suit_info(deets,ind):
    tmp_mal_rows=pd.DataFrame(columns=df.columns)
    for d in range(1,len(deets)):
        suit_info={}
        date=re.search('(..-..-....)', deets[d])
        suit_info['date'] = date.group()
        # area might be zip code or County, State
        try:
            area=re.search('\d{5}',deets[d].strip())
            suit_info['area'] = area.group()
            suit_info['significance']= deets[d].strip()[area.span()[1]+1:]
        except:
            try:
                _,end=date.span()
                area_obj=re.search('[A-Za-z]* *[\w]*(, NY)',deets[d][end:].strip())
                suit_info['area']=area_obj.group().strip()
                suit_info['significance']=deets[d][end:].strip()[area_obj.span()[1]+1:]
            except:
                _,end=date.span()
                area_obj=re.search('Not Available',deets[d][end:].strip())
                suit_info['area']=area_obj.group().strip()
                suit_info['significance']=deets[d][end:].strip()[area_obj.span()[1]+1:]
        tmp_row=pd.DataFrame([df.loc[ind]],columns=df.columns)
        tmp_row['date']=suit_info['date']
        tmp_row['area']=suit_info['area']
        tmp_row['significance']=suit_info['significance']
        tmp_mal_rows=pd.concat([tmp_mal_rows, tmp_row], axis=0)
    return tmp_mal_rows

In [14]:
# Fill in Law Suit info!
# creates empty data frame to fill
mal_rows=pd.DataFrame(columns=df.columns)
for ind in range(len(df)):
    #print(ind)
    mal=df.Malpractice[ind]
    if mal=='None reported':
        continue
    start,end =re.search('High   |   Low\n',mal).span()
    start2,end2=re.search("\n\nThe doctor\'s malpractice history",mal).span()
    deets=mal[end:start2].strip()
    deets=deets.split('\n')
    if re.search('None reported',deets[1]) != None:
        x,y=re.search('\n(..-..-....)',mal).span()
        x2,y2=re.search('\n\nSettlements',mal).span()
        deets=[0,mal[x+1:x2]]
    tmp_mal_rows=extract_law_suit_info(deets,ind)
    mal_rows=pd.concat([mal_rows, tmp_mal_rows], axis=0)
bueno_rows=df[df.Malpractice=="None reported"]
new_df=pd.concat([bueno_rows,mal_rows],axis=0)

In [15]:
new_df

,Malpractice,Name,Field,School,Year,State,Country,Num_Payments,ind_law_suit,date,area,significance
1,None reported,"ABU-RUSTUM, NADEEM",Obstetrics and Gynecology/Gynecologic Oncology,"AMERICAN UNIVERSITY OF BEIRUT, FAC OF MEDICINE...",1990,NaN,LEBANON,0,0,0,0,0
2,None reported,"AGUSTIN, ELMER",Obstetrics and Gynecology,"UNIVERSITY OF THE PHILIPPINES, COLLEGE OF MEDI...",1980,NaN,PHILIPPINES,0,0,0,0,0
4,None reported,"AL-IBRAHEEMI, ZAINAB",Obstetrics and Gynecology/Maternal-Fetal Medicine,"UNIVERSITY OF BAGHDAD, COLLEGE OF MEDICINE, BA...",2006,NaN,IRAQ,0,0,0,0,0
5,None reported,"ALLMEN, TARA",Obstetrics and Gynecology,"STONY BROOK UNIVERSITY HEALTH SCIENCES CENTER,...",1990,NY,USA,0,0,0,0,0
6,None reported,"AMARU, REBECCA",Obstetrics and Gynecology,"ICAHN SCH OF MED AT MT SINAI, NEW YORK, NY",1999,NY,USA,0,0,0,0,0
7,None reported,"AMRANE, SELMA",Obstetrics and Gynecology/Reproductive Endocri...,"UNIVERSITY OF MARYLAND SCHOOL OF MEDICINE, BAL...",2013,MD,USA,0,0,0,0,0
8,None reported,"ANDERSON, LISA",Obstetrics and Gynecology/Gynecologic Oncology,"UNIVERSITY DI BOLOGNA, FAC DI MED E CHIRURGIA,...",1980,NaN,ITALY,0,0,0,0,0
10,None reported,"AQUINO, JENNIFER",Obstetrics and Gynecology,"SUNY, DWNSTATE M C COLL MED, BROOKLYN NY",2012,NY,USA,0,0,0,0,0
11,None reported,"ARCHIBALD, CARMIT",Obstetrics and Gynecology,"YALE UNIVERSITY SCHOOL OF MEDICINE, NEW HAVEN CT",2001,CT,USA,0,0,0,0,0
12,None reported,"ARGOV, YURI",Obstetrics and Gynecology,"THE HEBREW UNIVERSITY, HADASSAH MEDICAL SCHOOL...",1980,NaN,ISRAEL,0,0,0,0,0
